In [1]:
# install ktrain on Google Colab
!pip3 install ktrain

     |████████████████████████████████| 25.2MB 129kB/s 
     |████████████████████████████████| 421.8MB 28kB/s 
     |████████████████████████████████| 6.7MB 42.1MB/s 
     |████████████████████████████████| 983kB 42.4MB/s 
     |████████████████████████████████| 245kB 40.5MB/s 
     |████████████████████████████████| 675kB 39.5MB/s 
     |████████████████████████████████| 471kB 37.6MB/s 
     |████████████████████████████████| 450kB 38.3MB/s 
     |████████████████████████████████| 3.9MB 39.2MB/s 
     |████████████████████████████████| 890kB 38.7MB/s 
     |████████████████████████████████| 1.1MB 29.3MB/s 
     |████████████████████████████████| 3.8MB 40.5MB/s 
  Created wheel for ktrain: filename=ktrain-0.16.3-cp36-none-any.whl size=25246180 sha256=22893d3d731dff54797ea507686a96191f453480a882d743952aee7fe681260d
  Stored in directory: /root/.cache/pip/wheels/2b/24/21/a52cc4dcb0b438cf570e9d44c017761d41335d9af4c189ebad
  Created wheel for keras-bert: filename=keras_bert-0.84.0-cp36-no

In [2]:
# import ktrain and the ktrain.text modules
import ktrain
from ktrain import text

In [3]:
ktrain.__version__

'0.16.3'

# Multiclass Text Classification Using BERT and Keras
In this example, we will use ***ktrain*** ([a lightweight wrapper around Keras](https://github.com/amaiya/ktrain)) to build a model using the dataset employed in the **scikit-learn** tutorial: [Working with Text Data](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html).  As in the tutorial, we will sample 4 newsgroups to create a relatively small multiclass text classification dataset.  The objective is to accurately classify each document into one of these four newsgroup topic categories.  This will provide us an opportunity to see **BERT** in action on a relatively smaller training set.  Let's fetch the [20newsgroups dataset ](http://qwone.com/~jason/20Newsgroups/) using scikit-learn.

In [4]:
# fetch the dataset using scikit-learn
categories = ['alt.atheism', 'soc.religion.christian',
             'comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
train_b = fetch_20newsgroups(subset='train',
   categories=categories, shuffle=True, random_state=42)
test_b = fetch_20newsgroups(subset='test',
   categories=categories, shuffle=True, random_state=42)

print('size of training set: %s' % (len(train_b['data'])))
print('size of validation set: %s' % (len(test_b['data'])))
print('classes: %s' % (train_b.target_names))

x_train = train_b.data
y_train = train_b.target
x_test = test_b.data
y_test = test_b.target

size of training set: 2257
size of validation set: 1502
classes: ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']


## STEP 1:  Load and Preprocess the Data
Preprocess the data using the `texts_from_array function` (since the data resides in an array).
If your documents are stored in folders or a CSV file you can use the `texts_from_folder` or `texts_from_csv` functions, respectively.

In [5]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       class_names=train_b.target_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

task: text classification
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


## STEP 2:  Load the BERT Model and Instantiate a Learner object

In [6]:
# you can disregard the deprecation warnings arising from using Keras 2.2.4 with TensorFlow 1.14.
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)

Is Multi-Label? False
maxlen is 350
done.


## STEP 3: Train the Model

We train using one of the three learning rates recommended in the BERT paper: *5e-5*, *3e-5*, or *2e-5*.
Alternatively, the ktrain Learning Rate Finder can be used to find a good learning rate by invoking `learner.lr_find()` and `learner.lr_plot()`, prior to training.
The `learner.fit_onecycle` method employs a [1cycle learning rate policy](https://arxiv.org/pdf/1803.09820.pdf).



In [7]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Train on 2257 samples
Epoch 1/4
2257/2257 [==============================] - 476s 211ms/sample - loss: 0.6438 - accuracy: 0.7554
Epoch 2/4
2257/2257 [==============================] - 455s 202ms/sample - loss: 0.1395 - accuracy: 0.9579
Epoch 3/4
2257/2257 [==============================] - 455s 202ms/sample - loss: 0.0374 - accuracy: 0.9885
Epoch 4/4
2257/2257 [==============================] - 455s 202ms/sample - loss: 0.0063 - accuracy: 0.9991


We can use the `learner.validate` method to test our model against the validation set.
As we can see, BERT achieves a **96%** accuracy, which is quite a bit higher than the 91% accuracy achieved by SVM in the [scikit-learn tutorial](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html).

In [8]:
learner.validate(val_data=(x_test, y_test), class_names=train_b.target_names)

                        precision    recall  f1-score   support

           alt.atheism       0.94      0.92      0.93       319
         comp.graphics       0.96      0.97      0.97       389
               sci.med       0.97      0.95      0.96       396
soc.religion.christian       0.94      0.98      0.96       398

              accuracy                           0.95      1502
             macro avg       0.95      0.95      0.95      1502
          weighted avg       0.95      0.95      0.95      1502



array([[292,   3,   6,  18],
       [  9, 377,   2,   1],
       [  5,  11, 375,   5],
       [  4,   1,   3, 390]])

## How to Use Our Trained BERT Model

We can call the `learner.get_predictor` method to obtain a Predictor object capable of making predictions on new raw data.

In [9]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [10]:
predictor.get_classes()

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [11]:
predictor.predict(test_b.data[0:1])

['sci.med']

In [12]:
# we can visually verify that our prediction of 'sci.med' for this document is correct
print(test_b.data[0])

From: brian@ucsd.edu (Brian Kantor)
Subject: Re: HELP for Kidney Stones ..............
Organization: The Avant-Garde of the Now, Ltd.
Lines: 12
NNTP-Posting-Host: ucsd.edu

As I recall from my bout with kidney stones, there isn't any
medication that can do anything about them except relieve the pain.

Either they pass, or they have to be broken up with sound, or they have
to be extracted surgically.

When I was in, the X-ray tech happened to mention that she'd had kidney
stones and children, and the childbirth hurt less.

Demerol worked, although I nearly got arrested on my way home when I barfed
all over the police car parked just outside the ER.
	- Brian



In [13]:
# we predicted the correct label
print(test_b.target_names[test_b.target[0]])

sci.med


The `predictor.save` and `ktrain.load_predictor` methods can be used to save the Predictor object to disk and reload it at a later time to make predictions on new data.

In [14]:
# let's save the predictor for later use
predictor.save('/tmp/my_predictor')

In [15]:
# reload the predictor
reloaded_predictor = ktrain.load_predictor('/tmp/my_predictor')

In [16]:
# make a prediction on the same document to verify it still works
reloaded_predictor.predict(test_b.data[0:1])

['sci.med']